**Question: Are there differences in medications prescribed off-label in patients diagnosed with Asthma during childhood vs during adulthood?**
* Find age of diagosis for Asthma patients
* Find medications (and RxNorm codes for meds) taken by Asthma patients
* Find all diagnoses of Asthma patients
* Classify medications taken by each patient as on- or off- label drug use
* Find medication classes for medications taken by Asthma patients
* Compare medication classes taken by patients diagosed with Asthma during childhood vs during adulthood

In [10]:
## All the imports we need
from urllib2 import Request, urlopen, URLError
from urllib import quote_plus

import mysql.connector

import pprint, json, requests

from datetime import datetime, timedelta
from dateutil.parser import parse as parse_date
from greentranslator.api import GreenTranslator

import dateutil

**Step 1: Find patients diagnosed with Asthma as a child (Group 1)**

In [11]:
## Find HUSH+ patients matching a list of ICD codes as children

query = GreenTranslator ().get_query ()

HUSHplusChildren = query.clinical_get_patients (age='8', sex='male', 
                                        race='white', location='OUTPATIENT')
##pprint.pprint (HUSHpluspatients)

Initializing thread provenance @thread: MainThread


**Step 2: Find patients diagnosed with Asthma as an adult (Group 2)**

In [12]:
## Find HUSH+ patients matching a list of ICD codes as adults

query = GreenTranslator ().get_query ()

HUSHplusAdults = query.clinical_get_patients (age='8', sex='male', 
                                        race='white', location='OUTPATIENT')
##pprint.pprint (HUSHpluspatients)

** Step 3: Find medications for patients in Group 1 and Group 2 **

In [14]:
## Pull meds for HUSH+ patients
meds = {}
for x in HUSHplusChildren:
    medList = x['medList']
    # Collect the unique meds
    for m in medList.keys():
        found = False
        try:
            meds[medList[m]] = meds[medList[m]]+1
        except KeyError:
            meds[medList[m]] = 1
pprint.pprint(meds)

{None: 4,
 u'24 HR Methylphenidate Hydrochloride 10 MG Extended Release Oral Capsule [Ritalin]': 1,
 u'ACYCLOVIR 5 % TOPICAL OINTMENT': 1,
 u'ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) SOLUTION FOR NEBULIZATION': 3,
 u'ALBUTEROL SULFATE HFA 90 MCG/ACTUATION AEROSOL INHALER': 7,
 u'AUGMENTIN ES-600 SUSPENSION': 3,
 u'AZITHROMYCIN 250 MG TABLET': 1,
 u'Acetaminophen 21.7 MG/ML / Hydrocodone Bitartrate 0.5 MG/ML Oral Solution': 1,
 u'Acetaminophen 32 MG/ML Oral Suspension': 1,
 u'Albuterol 0.83 MG/ML Inhalant Solution': 2,
 u'Amoxicillin 120 MG/ML / Clavulanate 8.58 MG/ML Oral Suspension': 2,
 u'Amoxicillin 80 MG/ML Oral Suspension': 8,
 u'Antipyrine 54 MG/ML / Benzocaine 14 MG/ML Otic Solution': 1,
 u'Azithromycin 40 MG/ML Oral Suspension': 10,
 u'BACTROBAN 2% OINTMENT': 1,
 u'BECLOMETHASONE DIPROPIONATE 40 MCG/ACTUATION AEROSOL INHALER': 3,
 u'CLARITHROMYCIN 250 MG TABLET': 1,
 u'CLEOCIN 75 MG/5 ML GRANULES': 1,
 u'CLONIDINE HCL ER 0.1 MG TABLET,EXTENDED RELEASE,12 HR': 4,
 u'CONCERTA ER 1

** Step 4: Find RxNorm codes for meds prescribed in patients from Group 1 and Group 2 **

In [3]:
## For a given medication string get NCBO annotations
## We let NCBO match any ontology since just using RxNORM doesn't
## always give us just the drug name (e.g., "CLINDAMYCIN 15 MG/ML ORAL SOLUTION" is
## a valid RxNORM term)
def med2rxnorm(txt):
    url = 'http://data.bioontology.org/annotator?text=%s&apikey=b792dd1b-cdc2-4cc8-aaf2-4fa4fbf47e4e'
    txt = urlopen(url % quote_plus(txt)).read()
    resp = json.loads(txt)
    if len(resp) == 0: return([])
    annos = []
    for aresp in resp:
        annos.extend([ x['text'] for x in aresp['annotations'] ])
    ##annos = filter(lambda x: not any(d in x for d in'0123456789'), annos)
    return(annos)
print med2rxnorm("CLINDAMYCIN 15 MG/ML ORAL SOLUTION")        

[u'CLINDAMYCIN 15 MG/ML ORAL SOLUTION', u'CLINDAMYCIN 15 MG/ML', u'CLINDAMYCIN', u'ORAL SOLUTION']


In [17]:
## Get RxNORM codes for medication strings
medrxnorm = {}
for med in meds.keys():
    if med is None: continue    
    annos = med2rxnorm(med)
    print 'Processing %s and found %d annotations' % (med, len(annos))
    medrxnorm[med] = {'count':meds[med], 'annos':annos}    

Processing TRIAMCINOLONE ACETONIDE 0.1 % TOPICAL OINTMENT and found 4 annotations
Processing Azithromycin 40 MG/ML Oral Suspension and found 4 annotations
Processing Trazodone Hydrochloride 50 MG Oral Tablet and found 5 annotations
Processing CONCERTA ER 27 MG TABLET and found 1 annotations
Processing NEOSTIGMINE METHYLSULFATE 0.5 MG/ML INJECTION SOLUTION and found 4 annotations
Processing CONCERTA ER 18 MG TABLET and found 1 annotations
Processing moxifloxacin 5 MG/ML Ophthalmic Solution and found 4 annotations
Processing BECLOMETHASONE DIPROPIONATE 40 MCG/ACTUATION AEROSOL INHALER and found 3 annotations
Processing Ofloxacin 3 MG/ML Otic Solution and found 4 annotations
Processing Diphenhydramine Hydrochloride 25 MG Oral Capsule and found 5 annotations
Processing Methylphenidate Hydrochloride 10 MG Oral Tablet and found 5 annotations
Processing ALBUTEROL SULFATE HFA 90 MCG/ACTUATION AEROSOL INHALER and found 3 annotations
Processing PROAIR HFA 90 MCG INHALER and found 1 annotations
P

** Step 5: Find all indications (as ICD-10 codes) for any of the drugs prescribed in patients from Group 1 and Group 2 **

In [15]:
## Given a drug (identified by RXNORM), get conditions that the drug has a 
## status (approved, phase 3) for, via NDK API
def drug2conditions(drug):
    import urllib
    url = "https://tripod.nih.gov/ndk/treatment/%s/conditions" % (drug)
    page = urllib.urlopen(url).read().strip()
    if page == "": return []
    resp = json.loads(page)
    conds = []
    for aresp in resp:
        condname = aresp['name']
        if 'ICD10' in aresp.keys():
            condicd10 = aresp['ICD10']
        else: condicd10 = []
        conds.append( (aresp['status'], condname, condicd10) )
    return(conds)

In [18]:
keys = list(medrxnorm.keys())
n = 10
annos = medrxnorm[keys[n]]['annos']
conds = []
for anno in annos:
    conds.extend(drug2conditions(anno))
print "##", keys[n], "##", conds

## Methylphenidate Hydrochloride 10 MG Oral Tablet ## [(u'Approved', u'Attention-deficit hyperactivity disorders', [u'F90']), (u'Approved', u'Syndrome', []), (u'Approved', u'Narcolepsy', [u'G47.41']), (u'Approved', u'Emotional lability', [u'R45.86']), (u'Approved', u'Developmental disorder of scholastic skills, unspecified', [u'F81.9']), (u'Approved', u'CHILD syndrome', []), (u'Approved', u'Mental disorder, not otherwise specified', [u'F99']), (u'Approved', u'Child', []), (u'Approved', u'Brief reactive psychosis', []), (u'Phase 4', u'Healthy', []), (u'Phase 4', u'Attention-deficit hyperactivity disorders', [u'F90']), (u'Phase 4', u'Dementia', []), (u'Phase 4', u'Attention-deficit/Hyperactivity Disorder', []), (u'Phase 4', u'Intracranial injury', [u'S06']), (u'Phase 4', u'Epilepsy', []), (u'Phase 4', u'Carboxylesterase 1 (CES1) Genotype', []), (u'Phase 4', u"Parkinson's disease", [u'G20']), (u'Phase 4', u'Oppositional defiant disorder', [u'F91.3']), (u'Phase 4', u'Attention Deficit/Hype

In [6]:
## Get ICD-10 codes for any med seen for Group 1 or Group 2

** Step 6: Find all diagnoses of Asthma patients in Group 1 and Group 2**

In [ ]:
## Find all diagnoses of HUSH+ patients in Group 1
## Find all diagnoses of HUSH+ patients in Group 2

**Step 7: For both groups, classify medications taken by each patient as on- or off- label drug use (patient by drug matrix)**

In [ ]:
## Create patient by drug matrix

** Step 8: Create patient by drug class matrix **

In [ ]:
## Create patient by drug class matrix

**Step 9: Assess off-label drug use in childhood-onset vs adult-onset Asthma patients **

In [8]:
## Compare patient x drug class for Group 1 vs Group 2